# Build a Retrieval Augmented Generation (RAG) App 构建检索增强生成

LLM 支持的最强大的应用程序之一是复杂的问答 (Q&A) 聊天机器人。  
这些应用程序可以回答有关特定源信息的问题。这些应用程序使用一种称为检索增强生成 (RAG) 的技术。

本教程将展示如何基于文本数据源构建一个简单的问答应用程序。在此过程中，我们将介绍典型的问答架构，并重点介绍更多高级问答技术的额外资源。  

## 什么是 RAG？
RAG 是一种通过附加数据增强 LLM 知识的技术。

LLM 可以推理广泛的主题，但它们的知识仅限于在训练它们的特定时间点之前的公共数据。  
如果您想构建能够推理私有数据或模型截止日期后引入的数据的 AI 应用程序，则需要使用模型所需的特定信息来增强模型的知识。将适当的信息插入模型提示的过程称为检索增强生成 (RAG)。

LangChain 有许多组件旨在帮助构建问答应用程序，以及更普遍的 RAG 应用程序。

注意：这里我们专注于非结构化数据的问答。

## 概念
典型的 RAG 应用程序有两个主要组件：

 - 索引：从源中提取数据并对其进行索引的管道。这通常是离线进行的。

 - 检索和生成：实际的 RAG 链，它在运行时接受用户查询并从索引中检索相关数据，然后将其传递给模型。

从原始数据到答案的最常见完整序列如下所示：

### 索引
 - 加载：首先我们需要加载数据。这可以通过 DocumentLoaders 完成。
 - 拆分：文本拆分器将大型文档拆分成较小的块。这对于索引数据和将其传递给模型都很有用，因为大块数据更难搜索，并且不适合模型的有限上下文窗口。
 - 存储：我们需要一个地方来存储和索引我们的拆分，以便以后可以搜索它们。这通常使用 VectorStore 和 Embeddings 模型来完成。

### 检索和生成
 - 检索：给定用户输入，使用检索器从存储中检索相关分割。
 - 生成：ChatModel/LLM 使用包含问题和检索到的数据的提示生成答案


让我们一步一步地分析上面的步骤。

## 1. 索引：加载 （Indexing: Load）
我们需要先加载网络文章内容。我们可以使用 DocumentLoaders 来实现这一点，它们是从源加载数据并返回文档列表的对象。Document 是一个包含一些 page_content (str) 和元数据 (dict) 的对象。

在这种情况下，我们将使用 WebBaseLoader，它使用 urllib 从 Web URL 加载 HTML，并使用 BeautifulSoup 将其解析为文本。我们可以通过 bs_kwargs 将参数传递给 BeautifulSoup 解析器来自定义 HTML -> 文本解析（请参阅 BeautifulSoup 文档）。在这种情况下，只有类为“post-content”、“post-title”或“post-header”的 HTML 标签才是相关的，因此我们将删除所有其他标签。

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(id=("article_content"))
loader = WebBaseLoader(
    web_paths=("https://ouzhoubei.co/article-794-1.html",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
len(docs[0].page_content)

8770

In [2]:
print(docs[0].page_content[:500])

【2024年欧洲杯小组赛A组】2024年欧洲杯德国队球员号码：1号：诺伊尔（拜仁慕尼黑）2号：吕迪格（皇家马德里）3号：劳姆（莱比锡）4号：塔（勒沃库森）5号：格罗斯（布莱顿）6号：基米希（拜仁慕尼黑）7号：哈弗茨（阿森纳）8号：克罗斯（皇马）9号：菲尔克鲁格（多特）10号：穆西亚拉（拜仁）11号：弗里希（斯图加特）12号：鲍曼（霍芬海姆）13号：穆勒（拜仁）14号：拜尔（霍芬海姆）15号：施洛特贝克（多特）16号：安东（斯图加特）17号：维尔茨（勒沃库森）18号：米特尔施塔特（斯图加特）19号：萨内（拜仁）20号：亨里希斯（莱比锡）21号：京多安（巴萨）22号：特尔施特根（巴塞罗那）23号：安德里希（勒沃库森）24号：科赫（法兰克福）25号：帕夫洛维奇（拜仁）26号：翁达夫（斯图加特）2024年欧洲杯瑞士球员号码：1号：索默（国米）2号：斯特吉奥（斯图加特）3号：威德默（美因茨）4号：埃尔维迪（门兴）5号：阿坎吉（曼城）6号：扎卡里亚（摩纳哥）7号：恩博洛（摩纳哥）8号：弗罗伊勒（博洛尼亚）9号：奥卡福（AC米兰）10号：扎卡（勒沃库森）11号：斯特芬（卢加诺）12号：姆沃戈（洛里昂


## 2. 索引：拆分 ( Indexing: Split)
我们加载的文档长度超过 8k 个字符。这太长了，无法放入许多模型的上下文窗口中。即使对于那些可以在其上下文窗口中容纳完整帖子的模型，模型也很难在非常长的输入中找到信息。

为了解决这个问题，我们将文档拆分成块以进行嵌入和向量存储。这应该有助于我们在运行时仅检索博客文章中最相关的部分。

在这种情况下，我们将文档拆分成 1000 个字符的块，块之间有 200 个字符的重叠。重叠有助于减轻将语句与与其相关的重要上下文分离的可能性。  
我们使用 RecursiveCharacterTextSplitter，它将使用常用分隔符（如换行符）递归拆分文档，直到每个块的大小合适。这是针对一般文本用例的推荐文本拆分器。

我们设置 add_start_index=True，以便每个拆分文档在初始文档中开始的字符索引保留为元数据属性“start_index”。

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

12

In [4]:
len(all_splits[0].page_content)

1000

In [5]:
all_splits[10].metadata

{'source': 'https://ouzhoubei.co/article-794-1.html', 'start_index': 7486}

## 3. 索引：存储 (Indexing: Store)
现在我们需要索引 66 个文本块，以便我们可以在运行时搜索它们。最常见的方法是嵌入每个文档拆分的内容，并将这些嵌入插入到向量数据库（或向量存储）中。  
当我们想要搜索我们的拆分时，我们会进行文本搜索查询，嵌入它，并执行某种“相似性”搜索，以识别与我们的查询嵌入最相似的嵌入的存储拆分。  
最简单的相似性度量是余弦相似性 - 我们测量每对嵌入（高维向量）之间角度的余弦。

我们可以使用 Chroma 向量存储和 OpenAIEmbeddings 模型在单个命令中嵌入和存储所有文档拆分。

In [6]:
# 获取你的智谱 API Key
# 在当前文件下创建一个.env文件，将api-key复制进去，如ZHIPUAI_API_KEY = "api-key"

from dotenv import load_dotenv,find_dotenv
import os 
_ = load_dotenv(find_dotenv())

In [7]:
from langchain_chroma import Chroma
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

这样就完成了管道的索引部分。此时，我们有一个可查询的向量存储，其中包含博客文章的分块内容。给定一个用户问题，理想情况下，我们应该能够返回回答该问题的博客文章片段。

## 4. 检索和生成：检索 (Retrieval and Generation: Retrieve)
现在让我们编写实际的应用程序逻辑。我们想要创建一个简单的应用程序，它接受用户问题，搜索与该问题相关的文档，将检索到的文档和初始问题传递给模型，并返回答案。

首先，我们需要定义搜索文档的逻辑。LangChain 定义了一个 Retriever 接口，它包装了一个索引，可以根据字符串查询返回相关文档。

最常见的 Retriever 类型是 VectorStoreRetriever，它使用向量存储的相似性搜索功能来促进检索。任何 VectorStore 都可以通过 VectorStore.as_retriever() 轻松转换为 Retriever：

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("意大利队法乔利身披几号球衣?")

len(retrieved_docs)

6

In [9]:
print(retrieved_docs[0].page_content)

：尤克塞克（费内巴切）17号：卡维奇（费内巴切）18号：穆尔杜（费内巴切）19号：伊尔迪兹（尤文图斯）20号：卡迪奥卢（费内巴切）21号：伊尔马兹（加拉塔萨雷）22号：艾汗（加拉塔萨雷）23号：恰基尔（特拉布宗体育）24号：基里索（贝西克塔斯）25号：阿克昆（加拉塔萨雷）26号：耶尔德勒姆（雷恩）2024年欧洲杯格鲁吉亚球员号码：1号：洛里亚（第比利斯迪纳摩）2号：卡卡巴泽（克拉科夫）3号：德瓦利（希腊人竞技）4号：卡希亚（斯洛万）5号：克维尔克维利亚（阿科多）6号：科乔拉什维利（莱万特）7号：克瓦拉茨赫利亚（那不勒斯）8号：齐夫齐瓦泽（卡尔斯鲁厄）9号：达维塔什维利（波尔多）10号：查克韦塔泽（沃特福德）11号：克维利泰亚（希腊人竞技）12号：古格沙什维利（卡拉巴赫）13号：戈乔列什维利（顿涅茨克矿工）14号：洛乔什维利（克雷莫纳）15号：格维列西亚尼（波斯波利斯）16号：克维克韦斯基里（波兹南莱赫）17号：基特希什维利（斯图姆）18号：阿尔图纳什维利（沃尔夫斯贝格）19号：申格利亚（帕纳托利科斯）20号：梅克瓦比什维利（克拉约瓦）21号：齐泰什维利（第比利斯迪纳摩）22号：米卡乌塔泽（梅斯）23号：洛布贾尼泽（亚特兰大联）24号：塔比泽（帕纳托利科斯）25号：玛玛达什维利（瓦伦西亚）26号：西格亚（巴塞尔）2024年欧洲杯葡萄牙球员号码：1号：帕特里西奥（罗马）2号：塞梅多（狼队）3号：佩佩（波尔图）4号：迪亚斯（曼城）5号：达洛特（曼联）6号：帕利尼亚（富勒姆）7号：C罗（利雅得胜利）8号：B费（曼联）9号：贡萨洛·拉莫斯（巴黎）10号：B席（曼城）11号：菲利克斯（巴萨）12号：若泽·萨（狼队）13号：达尼洛·佩雷拉（巴黎）14号：伊纳西奥（葡体）15号：若昂·内维斯（本菲卡）16号：努内斯（曼城）17号：莱奥（AC米兰）18号：鲁本·内维斯（利雅得新月）19号：门德斯（巴黎）20号：坎塞洛（巴萨）21号：若塔（利物浦）22号：迪奥戈·科斯塔（波尔图）23号：维蒂尼亚（巴黎）24号：安东尼奥·席尔瓦（本菲卡）25号：内托（狼队）26号：孔塞桑（波尔图）2024年欧洲杯捷克球员号码：1号：斯塔涅克（布拉格斯拉维亚）2号：齐马（布拉格斯拉维亚）3号：霍莱什（布拉格斯拉维亚）4号：赫拉纳克（比尔森胜利）5号：曹法尔（西汉姆）6号：维蒂克（布拉格斯巴达）7号：巴拉克

## 5. 检索和生成：生成(Retrieval and Generation: Generate)
让我们将所有内容组合成一个链条，该链条接收问题、检索相关文档、构造提示、将其传递给模型并解析输出。

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

我们将使用签入 LangChain 提示中心的 RAG 提示

In [11]:
# 安装langchainhub
# pip install langchainhub

In [20]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

我们将使用 LCEL Runnable 协议来定义链，使我们能够

 - 以透明的方式将组件和功能连接在一起

 - 开箱即用地实现流式、异步和批量调用。


以下是实现：

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("意大利队法乔利身披几号球衣?"):
    print(chunk, end="", flush=True)

根据提供的上下文，意大利队法乔利身披的球衣号码是21号。